In [ ]:
# --- 1. MOUNT DRIVE & INSTALL LIBRARY ---
import os
import shutil
from google.colab import drive

# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Install Library (Hanya perlu sekali di awal sesi)
print("⏳ Sedang menginstall library... (Tunggu 2-3 menit)")
!pip install -q git+https://github.com/huggingface/diffusers.git
!pip install -q -U -r https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/requirements.txt
!pip install -q bitsandbytes peft accelerate transformers

# Download script training
!wget -q -nc https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora.py

print("✅ Setup Selesai! Sekarang kamu bisa memanggil fungsi training.")

Mounted at /content/drive
⏳ Sedang menginstall library... (Tunggu 2-3 menit)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.3 MB/s eta 0:0

In [ ]:
import os
import json
import math
from IPython import get_ipython

def fix_metadata_and_train_archive2_save_best(epochs=150, batch_size=6):
    # --- KONFIGURASI ---
    FOLDER_UTAMA = "archive"
    DATASET_DIR = f"/content/drive/MyDrive/DL_Finalisasi/dataset/{FOLDER_UTAMA}"
    OUTPUT_DIR = f"/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"
    MODEL_BASE = "runwayml/stable-diffusion-v1-5"

    print(f"\n{'='*50}")
    print(f"🚑 PERBAIKAN METADATA & TRAINING (SAVE EVERY EPOCH)")
    print(f"{'='*50}\n")

    if not os.path.exists(DATASET_DIR):
        print(f"❌ Error: Folder {DATASET_DIR} tidak ditemukan!")
        return

    # --- 1. HAPUS METADATA LAMA ---\n
    metadata_path = os.path.join(DATASET_DIR, "metadata.jsonl")
    if os.path.exists(metadata_path):
        os.remove(metadata_path)
        print("🗑️ Metadata lama dihapus.")

    # --- 2. BUAT METADATA BARU ---
    print("📝 Sedang menulis ulang metadata.jsonl...")
    metadata = []
    valid_extensions = ('.png', '.jpg', '.jpeg', '.webp')
    count = 0

    for root, dirs, files in os.walk(DATASET_DIR):
        if ".ipynb_checkpoints" in root: continue
        folder_name = os.path.basename(root)
        if folder_name == FOLDER_UTAMA: continue

        # Format caption: batik_style_namabatik
        style_name = folder_name.replace("batik-", "").replace("batik_", "").replace("-", "_")
        caption = f"batik_style_{style_name}"

        for filename in files:
            if filename.lower().endswith(valid_extensions):
                full_path = os.path.join(root, filename)
                rel_path = os.path.relpath(full_path, DATASET_DIR)
                metadata.append({"file_name": rel_path, "text": caption})
                count += 1

    if count == 0:
        print("❌ Tidak ada gambar ditemukan!")
        return

    with open(metadata_path, 'w') as f:
        for entry in metadata:
            json.dump(entry, f)
            f.write('\n')

    print(f"✅ Metadata BARU siap! Total: {count} gambar.")

    # --- 3. HITUNG STEPS UNTUK 'SAVE BEST' STRATEGY ---
    # Kita ingin save SETIAP EPOCH agar tidak kehilangan momen terbaik
    # Rumus: Jumlah Gambar / Batch Size = 1 Epoch
    steps_per_epoch = math.ceil(count / batch_size)

    print(f"📊 Info Training:")
    print(f"   - Total Gambar: {count}")
    print(f"   - Batch Size: {batch_size}")
    print(f"   - Steps per Epoch: {steps_per_epoch} steps")
    print(f"   - Total Epoch: {epochs}")
    print(f"   👉 Model akan disimpan setiap {steps_per_epoch} steps (Tiap Epoch)")

    print(f"\n🚀 MEMULAI TRAINING...")

    # Prompt validasi untuk mengecek kualitas setiap epoch
    # Ganti 'batik_style_parang' dengan salah satu style kamu jika mau spesifik
    val_prompt = "batik_style_parang"

    command = (
        f"accelerate launch train_text_to_image_lora.py "
        f"--pretrained_model_name_or_path='{MODEL_BASE}' "
        f"--train_data_dir='{DATASET_DIR}' "
        f"--caption_column='text' "
        f"--resolution=512 "
        f"--random_flip "
        f"--train_batch_size={batch_size} "
        f"--gradient_accumulation_steps=1 "
        f"--num_train_epochs={epochs} "
        # KUNCI SAVE BEST: Simpan setiap kali 1 epoch selesai
        f"--checkpointing_steps={steps_per_epoch} "
        # VALIDASI: Generate gambar setiap epoch supaya bisa dipantau
        f"--validation_prompt='{val_prompt}' "
        f"--validation_epochs=1 "
        f"--learning_rate=1e-04 "
        f"--lr_scheduler='cosine' "
        f"--lr_warmup_steps=0 "
        f"--seed=42 "
        f"--output_dir='{OUTPUT_DIR}' "
        f"--mixed_precision='fp16' "
        f"--rank=128 "
        # Opsional: Hapus checkpoint lama otomatis (simpan 5 terbaik terakhir)
        # f"--checkpoints_total_limit=5 "
    )

    get_ipython().system(command)
    print(f"\n🎉 TRAINING SELESAI!")
    print(f"Cek folder: {OUTPUT_DIR}")
    print("Di sana akan ada banyak folder 'checkpoint-XXX'.")
    print("Gunakan checkpoint dengan nomor langkah (steps) dimana hasil validasi terlihat paling bagus.")

# Jalankan dengan konfigurasi baru (150 epoch)
fix_metadata_and_train_archive2_save_best(epochs=130, batch_size=28)


🚑 PERBAIKAN METADATA & TRAINING (SAVE EVERY EPOCH)

🗑️ Metadata lama dihapus.
📝 Sedang menulis ulang metadata.jsonl...
✅ Metadata BARU siap! Total: 560 gambar.
📊 Info Training:
   - Total Gambar: 560
   - Batch Size: 28
   - Steps per Epoch: 20 steps
   - Total Epoch: 130
   👉 Model akan disimpan setiap 20 steps (Tiap Epoch)

🚀 MEMULAI TRAINING...
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-12-17 02:07:34.790999: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive

# --- 0. MOUNT DRIVE (WAJIB SETELAH RESTART) ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- 1. KONFIGURASI ---
# Pastikan folder checkpoint ini BENAR-BENAR ADA di Google Drive kamu.
# Cek di folder file di kiri colab. Kalau training sampai 1600, pakai 1600.
LORA_PATH = "/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# Masukkan 10 jenis batik (atau prompt) yang mau dites
PROMPTS_LIST = [
    "batik_style_parang", "batik_style_megamendung", "batik_style_kawung", "batik_style_sekar_jagad", "batik_style_tambal",
    "batik_style_sidomukti", "batik_style_truntum", "batik_style_lasem", "batik_style_tujuh_rupa", "batik_style_priyangan"
]

def setup_pipeline():
    print("⏳ Loading Stable Diffusion & LoRA...")
    pipe = StableDiffusionPipeline.from_pretrained(
            BASE_MODEL,
            torch_dtype=torch.float16,
            safety_checker=None,  # <--- INI KUNCINYA
            feature_extractor=None # Perlu diset None juga jika safety_checker None
        ).to("cuda")

    # Cek dulu apakah path LoRA valid sebelum loading
    if not os.path.exists(LORA_PATH):
        raise FileNotFoundError(f"❌ Path LoRA tidak ditemukan: {LORA_PATH}\nSilakan cek folder Drive dan ganti angka checkpoint-nya.")

    pipe.load_lora_weights(LORA_PATH)

    print("⏳ Loading CLIP Model (Juri Penilai)...")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    return pipe, clip_model, clip_processor

def calculate_clip_score(image, text, model, processor):
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True).to("cuda")
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    return logits_per_image.item()

def generate_and_evaluate():
    pipe, clip_model, clip_processor = setup_pipeline()

    # Setup Plot 2 Baris x 5 Kolom
    fig, axes = plt.subplots(2, 5, figsize=(20, 9))
    check_name = LORA_PATH.split("/")[-1]
    fig.suptitle(f'Hasil Test Model LoRA Batik ({check_name})', fontsize=16)

    axes = axes.flatten()

    print("\n🚀 Mulai Generate 10 Gambar...")

    for i, prompt in enumerate(PROMPTS_LIST):
        if i >= 10: break

        # Generate Image
        image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

        # Hitung Score
        score = calculate_clip_score(image, prompt, clip_model, clip_processor)

        # Tampilkan
        ax = axes[i]
        ax.imshow(image)
        ax.axis("off")

        short_name = prompt.replace("batik_style_", "").title()
        color = "green" if score > 30 else "black"
        ax.set_title(f"{short_name}\nCLIP Score: {score:.2f}", color=color, fontsize=10, fontweight='bold')

        print(f"✅ [{i+1}/10] {short_name} - Score: {score:.2f}")

    plt.tight_layout()
    plt.show()

# Jalankan
generate_and_evaluate()

In [ ]:
# ... (kode setup_pipeline dan calculate_clip_score di atas TETAP SAMA) ...

def generate_and_evaluate_v2():
    pipe, clip_model, clip_processor = setup_pipeline()

    # --- SETTING PROMPT BARU ---
    # Tambahan detail agar gambar lebih bagus (Suffix)
    additional_prompt = ", close up photo of batik fabric, intricate pattern, high quality texture, 8k, sharp focus"

    # Negative prompt untuk membuang hasil jelek
    neg_prompt = "blurry, low quality, pixelated, ugly, watermark, text, distortion, messy"

    # Setup Plot
    fig, axes = plt.subplots(2, 5, figsize=(20, 10)) # Sedikit diperbesar
    fig.suptitle(f'Hasil Test Model LoRA Batik (Improved Prompts)', fontsize=16)
    axes = axes.flatten()

    print("\n🚀 Mulai Generate 10 Gambar dengan Prompt yang Lebih Bagus...")

    for i, trigger_word in enumerate(PROMPTS_LIST):
        if i >= 10: break

        # Gabungkan Trigger Word + Tambahan
        final_prompt = f"{trigger_word}{additional_prompt}"

        # Generate Image (Perhatikan penambahan negative_prompt)
        image = pipe(
            final_prompt,
            negative_prompt=neg_prompt,
            num_inference_steps=35,  # Naikkan dikit biar lebih matang
            guidance_scale=8.0,      # Naikkan dikit agar lebih patuh pada prompt
            width=512,
            height=512
        ).images[0]

        # Hitung Score (pakai trigger word asli saja untuk penilaian)
        score = calculate_clip_score(image, trigger_word, clip_model, clip_processor)

        # Tampilkan
        ax = axes[i]
        ax.imshow(image)
        ax.axis("off")

        short_name = trigger_word.replace("batik_style_", "").title()
        color = "green" if score > 28 else "black" # Batas aman score

        # Tampilkan Nama Batik & Score
        ax.set_title(f"{short_name}\nCLIP: {score:.2f}", color=color, fontsize=11, fontweight='bold')
        print(f"✅ [{i+1}/10] {short_name} - Score: {score:.2f}")

    plt.tight_layout()
    plt.show()

# Jalankan versi baru
generate_and_evaluate_v2()

In [ ]:
def generate_clothing_test():
    pipe, clip_model, clip_processor = setup_pipeline()

    # Kita tes 5 motif teratas saja biar tidak terlalu lama
    selected_styles = PROMPTS_LIST[:5]

    # Definisi Prompt untuk tiap jenis baju
    clothing_types = {
        "Hoodie": "streetwear hoodie with {}, studio lighting, high quality, casual style, 8k, sharp focus",
        "Dress": "beautiful woman wearing elegant long dress with {} pattern, fashion photography, runway model, intricate details, masterpiece",
        "Shirt": "men's formal batik shirt with {} motif, close up, professional look, realistic fabric texture, 8k"
    }

    # Negative prompt standar
    neg_prompt = "bad anatomy, ugly, deformed, blurry, low quality, pixelated, watermark, text, messy, cartoon, illustration"

    # Setup Plot (Baris = Jumlah Motif, Kolom = Jenis Baju)
    fig, axes = plt.subplots(len(selected_styles), 3, figsize=(15, 25))
    fig.suptitle(f'Test Implementasi Batik pada Pakaian', fontsize=16)

    print("\n🚀 Mulai Generate Baju...")

    for i, trigger_word in enumerate(selected_styles):
        short_name = trigger_word.replace("batik_style_", "").title()

        # Loop untuk setiap jenis baju (Kolom)
        for j, (item_name, prompt_template) in enumerate(clothing_types.items()):

            # Masukkan trigger word ke dalam template prompt
            final_prompt = prompt_template.format(trigger_word)

            # Generate
            image = pipe(
                final_prompt,
                negative_prompt=neg_prompt,
                num_inference_steps=35,
                guidance_scale=8.0, # Agak tinggi biar polanya nempel
                width=512,
                height=768 # Format potrait lebih cocok untuk baju
            ).images[0]

            # Tampilkan
            if len(selected_styles) > 1:
                ax = axes[i, j]
            else:
                ax = axes[j]

            ax.imshow(image)
            ax.axis("off")

            if i == 0: # Judul kolom hanya di baris pertama
                ax.set_title(f"{item_name}", fontsize=14, fontweight='bold', pad=10)

            # Label di sebelah kiri untuk nama batik
            if j == 0:
                ax.text(-0.1, 0.5, short_name, transform=ax.transAxes,
                        va='center', ha='right', fontsize=12, fontweight='bold', rotation=90)

        print(f"✅ Selesai baris: {short_name}")

    plt.tight_layout()
    plt.show()

# Jalankan
generate_clothing_test()

In [ ]:
# --- 1. INSTALL LIBRARY ---
!pip install -q torchmetrics

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from torchmetrics.multimodal.clip_score import CLIPScore
import os

# --- 2. SETUP MESIN SULTAN (REALISTIC VISION) ---
print("⏳ Menyiapkan Mesin Realistic Vision...")

base_model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
device = "cuda"

if 'pipe' not in globals():
    pipe = StableDiffusionPipeline.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
        safety_checker=None,
        requires_safety_checker=False
    ).to(device)

pipe.safety_checker = None
pipe.requires_safety_checker = False

# Setup CLIP Score
try:
    if 'metric' not in globals() or metric is None:
        metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    print("✅ CLIP Score siap.")
except:
    metric = None

# --- 3. KONFIGURASI LORA ---
# Masukkan Path Model LoRA Utamamu di sini
# (Asumsi: 1 Model LoRA ini bisa mengenali berbagai style jika ditrigger)
LORA_PATH = "/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"

if os.path.exists(LORA_PATH):
    print(f"✅ Loading LoRA: {LORA_PATH}")
    pipe.unload_lora_weights()
    pipe.load_lora_weights(LORA_PATH)
else:
    print(f"⚠️ PERINGATAN: Path {LORA_PATH} tidak ditemukan. Cek lagi path-nya!")

# --- 4. DAFTAR 10 BATIK & ITEM FLEXING ---
# Kita pasangkan 1-lawan-1 biar variatif
daftar_flexing = [
    {
        "style": "Megamendung",
        "trigger": "batik_style_megamendung",
        "item": "Sports Car",
        "prompt": "Wide angle shot of a luxury sports car wrapped in {trigger} pattern vinyl decal, drifting on track, smoke, cinematic lighting, hyperrealistic, 8k"
    },
    {
        "style": "Parang",
        "trigger": "batik_style_parang",
        "item": "Luxury Handbag",
        "prompt": "RAW photo of a high-end leather handbag made of {trigger} fabric, intricate {trigger} pattern, expensive, studio lighting, product photography, 8k"
    },
    {
        "style": "Kawung",
        "trigger": "batik_style_kawung",
        "item": "Sneakers",
        "prompt": "Macro shot of limited edition sneakers made of {trigger} fabric, detailed embroidery, floating in mid-air, commercial photography, dramatic lighting, 8k"
    },
    {
        "style": "Sekar Jagad",
        "trigger": "batik_style_sekar_jagad",
        "item": "Modern Architecture",
        "prompt": "Architectural photography of a modern skyscraper facade with {trigger} pattern skin, glass and steel structure, sunlight, blue sky, futuristic design"
    },
    {
        "style": "Tambal",
        "trigger": "batik_style_tambal",
        "item": "Bomber Jacket",
        "prompt": "Medium shot of a cyberpunk street samurai wearing a bomber jacket with glowing {trigger} pattern neon texture, rainy neon city background, cinematic"
    },
    {
        "style": "Sidomukti",
        "trigger": "batik_style_sidomukti",
        "item": "Luxury Sofa",
        "prompt": "Interior photography of a modern luxury living room, expensive velvet sofa upholstered in {trigger} fabric design, marble floor, warm lighting"
    },
    {
        "style": "Truntum",
        "trigger": "batik_style_truntum",
        "item": "Origami Art",
        "prompt": "Macro shot of a complex origami dragon folded from {trigger} patterned paper, textured paper grain, depth of field, soft studio lighting"
    },
    {
        "style": "Lasem",
        "trigger": "batik_style_lasem",
        "item": "Porcelain Vase",
        "prompt": "Close up photo of an ancient chinese porcelain vase with painted {trigger} pattern, glossy texture, museum lighting, black background, 8k"
    },
    {
        "style": "Tujuh Rupa",
        "trigger": "batik_style_tujuh_rupa",
        "item": "Floral Dress",
        "prompt": "RAW photo of a fashion model wearing a flowing summer dress with {trigger} floral pattern, garden background, natural sunlight, bokeh, high fashion"
    },
    {
        "style": "Priyangan",
        "trigger": "batik_style_priyangan",
        "item": "Wall Decor",
        "prompt": "A modern minimalist bedroom with a feature wall covered in {trigger} wallpaper, textured paper, soft sunlight from window, cozy atmosphere"
    }
]

# --- 5. FUNGSI GENERATE 10 GAMBAR ---
def generate_10_flexing(seed=42):
    print(f"\n{'='*70}")
    print(f"💎 GENERATE 10 BATIK SULTAN (Seed: {seed})")
    print(f"{'='*70}")

    # Setup Canvas (2 Baris x 5 Kolom)
    fig, axs = plt.subplots(2, 5, figsize=(25, 12))
    axs = axs.flatten()

    # Negative Prompt
    negative_prompt = "(deformed, distorted, disfigured:1.3), cartoon, illustration, painting, drawing, anime, low quality, blurry, watermark, text, bad anatomy, ugly"

    for i, data in enumerate(daftar_flexing):
        print(f"🔄 ({i+1}/10) {data['style']} -> {data['item']}...", end="\r")

        try:
            # Format Prompt
            final_prompt = data['prompt'].format(trigger=data['trigger'])

            # Generate (Fixed Seed)
            generator = torch.Generator(device).manual_seed(seed)

            image = pipe(
                final_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=40,    # Detail tinggi
                guidance_scale=7.5,
                cross_attention_kwargs={"scale": 1.0}, # Full Power
                generator=generator
            ).images[0]

            # Hitung Score
            score = 0.0
            if metric:
                # Potong prompt 77 token
                text_input = final_prompt[:77]
                img_tensor = torch.from_numpy(np.array(image)).permute(2, 0, 1)
                score = metric(img_tensor, text_input).item()

            # Visualisasi
            ax = axs[i]
            ax.imshow(image)

            # Judul
            color = "green" if score > 27 else "black"
            title_text = f"{data['style']}\n({data['item']})\nScore: {score:.2f}"
            ax.set_title(title_text, fontsize=11, fontweight='bold', color=color)
            ax.axis("off")

        except Exception as e:
            print(f"\n❌ Error {data['style']}: {e}")
            axs[i].axis("off")

    plt.tight_layout()
    plt.show()
    print("\n✅ SELESAI! 10 Mahakarya Batik siap dipamerkan.")

# --- 6. EKSEKUSI ---
generate_10_flexing(seed=42)

In [ ]:
# --- 1. SETUP LIBRARY & MODEL ---
!pip install -q torchmetrics

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from torchmetrics.multimodal.clip_score import CLIPScore
import os

# --- 2. CONFIG: MODEL & LORA ---
# Gunakan Realistic Vision untuk hasil "Mahal"
BASE_MODEL = "SG161222/Realistic_Vision_V5.1_noVAE"
# Path LoRA (Single Model yang berisi 10 Style)
LORA_PATH = "/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"

print(f"⏳ Loading Base Model: {BASE_MODEL}...")
pipe = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    use_safetensors=True,
    safety_checker=None,
    requires_safety_checker=False
).to("cuda")

pipe.safety_checker = None # Matikan checker

# Load LoRA (Cukup sekali load karena isinya sudah 10 style)
if os.path.exists(LORA_PATH):
    print(f"✅ Loading LoRA Weights: {LORA_PATH}")
    pipe.load_lora_weights(LORA_PATH)
else:
    print(f"⚠️ PERINGATAN: Path LoRA tidak ditemukan! Pastikan path benar.")

# Setup CLIP Score
try:
    if 'metric' not in globals() or metric is None:
        metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    print("✅ CLIP Score siap.")
except:
    metric = None

# --- 3. DATASET: 10 MOTIF ---
TRIGGERS = [
    {"name": "Megamendung", "trigger": "batik_style_megamendung"},
    {"name": "Parang",      "trigger": "batik_style_parang"},
    {"name": "Kawung",      "trigger": "batik_style_kawung"},
    {"name": "Sekar Jagad", "trigger": "batik_style_sekar_jagad"},
    {"name": "Tambal",      "trigger": "batik_style_tambal"},
    {"name": "Sidomukti",   "trigger": "batik_style_sidomukti"},
    {"name": "Truntum",     "trigger": "batik_style_truntum"},
    {"name": "Lasem",       "trigger": "batik_style_lasem"},
    {"name": "Tujuh Rupa",  "trigger": "batik_style_tujuh_rupa"},
    {"name": "Priyangan",   "trigger": "batik_style_priyangan"}
]

# --- 4. TEMPLATE PROMPT (PILIH SALAH SATU) ---

# OPSI A: SPORTS CAR (Paling Keren buat Flexing)
PROMPT_TEMPLATE = "Wide angle shot of a luxury sports car wrapped in {trigger} pattern vinyl decal, drifting on track, smoke, cinematic lighting, hyperrealistic, 8k, automotive photography"
ITEM_NAME = "Luxury Sports Car"

# OPSI B: HANDBAG (Kalau mau ganti, uncomment baris di bawah ini)
# PROMPT_TEMPLATE = "RAW photo of a high-end leather handbag made of {trigger} fabric, intricate {trigger} pattern, expensive, studio lighting, product photography, 8k"
# ITEM_NAME = "Luxury Handbag"

# --- 5. FUNGSI EKSEKUSI ---
def compare_same_prompt(seed=42):
    print(f"\n{'='*70}")
    print(f"🏎️ TEST KONSISTENSI: {ITEM_NAME}")
    print(f"   Prompt yang sama akan diaplikasikan ke 10 Batik.")
    print(f"{'='*70}")

    # Setup Grid (2 Baris x 5 Kolom)
    fig, axs = plt.subplots(2, 5, figsize=(25, 10))
    axs = axs.flatten()

    # Negative Prompt
    negative_prompt = "(deformed, distorted, disfigured:1.3), cartoon, illustration, anime, low quality, blurry, watermark, text, bad anatomy, ugly"

    for i, batik in enumerate(TRIGGERS):
        print(f"🔄 ({i+1}/10) Wrapping {ITEM_NAME} dengan motif {batik['name']}...", end="\r")

        # 1. Siapkan Prompt
        final_prompt = PROMPT_TEMPLATE.format(trigger=batik['trigger'])

        # 2. Generate (Seed Terkunci = Posisi Mobil Sama Persis)
        generator = torch.Generator("cuda").manual_seed(seed)

        image = pipe(
            final_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=40,
            guidance_scale=7.5,
            cross_attention_kwargs={"scale": 1.0},
            generator=generator
        ).images[0]

        # 3. Hitung Score
        score = 0.0
        if metric:
            text_input = final_prompt[:77]
            img_tensor = torch.from_numpy(np.array(image)).permute(2, 0, 1)
            score = metric(img_tensor, text_input).item()

        # 4. Tampilkan
        ax = axs[i]
        ax.imshow(image)

        # Judul
        color = "green" if score > 27 else "black"
        ax.set_title(f"{batik['name']}\nScore: {score:.2f}", fontsize=12, fontweight='bold', color=color)
        ax.axis("off")

    plt.tight_layout()
    plt.suptitle(f"Perbandingan 10 Motif Batik pada {ITEM_NAME} (Seed: {seed})", fontsize=20, y=1.02)
    plt.show()
    print("\n✅ Selesai! Lihat bagaimana motif berubah tapi mobilnya tetap sama.")

# --- 6. JALANKAN ---
compare_same_prompt(seed=42)

In [ ]:
# --- 1. SETUP LIBRARY & MODEL ---
!pip install -q torchmetrics

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from torchmetrics.multimodal.clip_score import CLIPScore
import os

# --- 2. CONFIG: MODEL & LORA ---
# Gunakan Realistic Vision agar tasnya nyata, tapi dia juga jago bikin tekstur flat
BASE_MODEL = "SG161222/Realistic_Vision_V5.1_noVAE"
# Path LoRA (Single Model yang berisi 10 Style)
LORA_PATH = "/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"

print(f"⏳ Loading Base Model: {BASE_MODEL}...")
# Cek pipe agar tidak load berulang kali jika cell dijalankan ulang
if 'pipe' not in globals():
    pipe = StableDiffusionPipeline.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        use_safetensors=True,
        safety_checker=None,
        requires_safety_checker=False
    ).to("cuda")

pipe.safety_checker = None

# Load LoRA (Cukup sekali load di awal)
if os.path.exists(LORA_PATH):
    print(f"✅ Loading LoRA Weights: {LORA_PATH}")
    pipe.unload_lora_weights() # Unload dulu untuk memastikan bersih
    pipe.load_lora_weights(LORA_PATH)
else:
    print(f"⚠️ PERINGATAN: Path LoRA tidak ditemukan! Pastikan path benar.")

# Setup CLIP Score
try:
    if 'metric' not in globals() or metric is None:
        metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    print("✅ CLIP Score siap.")
except:
    metric = None

# --- 3. DATASET: 10 MOTIF BATIK ---
TRIGGERS = [
    {"name": "Megamendung", "trigger": "batik_style_megamendung"},
    {"name": "Parang",      "trigger": "batik_style_parang"},
    {"name": "Kawung",      "trigger": "batik_style_kawung"},
    {"name": "Sekar Jagad", "trigger": "batik_style_sekar_jagad"},
    {"name": "Tambal",      "trigger": "batik_style_tambal"},
    {"name": "Sidomukti",   "trigger": "batik_style_sidomukti"},
    {"name": "Truntum",     "trigger": "batik_style_truntum"},
    {"name": "Lasem",       "trigger": "batik_style_lasem"},
    {"name": "Tujuh Rupa",  "trigger": "batik_style_tujuh_rupa"},
    {"name": "Priyangan",   "trigger": "batik_style_priyangan"}
]

# --- 4. TEMPLATE PROMPT (DUA SKENARIO) ---

# SKENARIO 1: LUXURY BAG (3D Realistic)
PROMPT_BAG = "RAW photo of a high-end designer handbag made of {trigger} fabric, close up, intricate {trigger} texture, expensive leather trim, studio lighting, product photography, 8k uhd, sharp focus"
NEGATIVE_BAG = "(deformed, distorted, disfigured:1.3), cartoon, illustration, anime, low quality, blurry, watermark, text, bad anatomy, ugly"

# SKENARIO 2: FIGMA BACKGROUND (2D Flat & Seamless)
# Keyword penting: "seamless repetitive pattern", "flat lay", "UI design texture"
PROMPT_FIGMA = "A seamless repetitive background pattern for UI design figma texture consisting of {trigger} motif, flat lay, top down view, tiling texture, high resolution fabric pattern, clean style, no shadows"
NEGATIVE_FIGMA = "3d render, realistic photo, shadows, depth, perspective, deformed, blurry, low quality, watermark, text"


# --- 5. FUNGSI EKSEKUSI UTAMA ---
def run_comparison_grid(prompt_template, negative_prompt, item_name, seed=42):
    print(f"\n{'='*70}")
    print(f"📊 TEST KOMPARASI: {item_name.upper()} (Seed: {seed})")
    print(f"{'='*70}")

    # Setup Grid (2 Baris x 5 Kolom)
    fig, axs = plt.subplots(2, 5, figsize=(25, 10))
    axs = axs.flatten()

    for i, batik in enumerate(TRIGGERS):
        print(f"🔄 ({i+1}/10) Generating {item_name} dengan motif {batik['name']}...", end="\r")

        # 1. Siapkan Prompt
        final_prompt = prompt_template.format(trigger=batik['trigger'])

        # 2. Generate (Seed Terkunci agar adil)
        generator = torch.Generator("cuda").manual_seed(seed)

        # Kita gunakan scale 1.0 agar motifnya benar-benar 'nempel'
        image = pipe(
            final_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=35,
            guidance_scale=7.5,
            cross_attention_kwargs={"scale": 1.0},
            generator=generator
        ).images[0]

        # 3. Hitung Score
        score = 0.0
        if metric:
            text_input = final_prompt[:77]
            img_tensor = torch.from_numpy(np.array(image)).permute(2, 0, 1)
            score = metric(img_tensor, text_input).item()

        # 4. Tampilkan
        ax = axs[i]
        ax.imshow(image)

        # Judul
        # Ambang batas hijau/hitam disesuaikan sedikit per skenario
        threshold = 28 if "Bag" in item_name else 26
        color = "green" if score > threshold else "black"

        ax.set_title(f"{batik['name']}\nScore: {score:.2f}", fontsize=12, fontweight='bold', color=color)
        ax.axis("off")

    plt.tight_layout()
    plt.suptitle(f"Perbandingan 10 Motif Batik pada {item_name}", fontsize=20, y=1.02)
    plt.show()
    print(f"\n✅ Selesai untuk {item_name}!")


# --- 6. JALANKAN KEDUANYA ---

# RUN SKENARIO 1: Tas Mewah
run_comparison_grid(PROMPT_BAG, NEGATIVE_BAG, "Luxury Handbag", seed=42)

# RUN SKENARIO 2: Figma Background
# Kita pakai seed berbeda (misal 123) untuk figma agar variasi awalnya beda dengan tas
run_comparison_grid(PROMPT_FIGMA, NEGATIVE_FIGMA, "Figma Repetitive Background", seed=123)

In [ ]:
# --- 1. SETUP LIBRARY ---
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline

# --- 2. SETUP MODEL ---
BASE_MODEL = "SG161222/Realistic_Vision_V5.1_noVAE"
LORA_PATH = "/content/drive/MyDrive/COLAB/ModelLora_Batik_HigesthBatch"

print(f"⏳ Loading Model...")
if 'pipe' not in globals():
    pipe = StableDiffusionPipeline.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        safety_checker=None,
        requires_safety_checker=False
    ).to("cuda")

if 'metric' not in globals():
    try:
        from torchmetrics.multimodal.clip_score import CLIPScore
        metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    except: metric = None

pipe.unload_lora_weights()
pipe.load_lora_weights(LORA_PATH)

# --- 3. DICTIONARY "PENYELAMAT" (VISUAL BOOSTER) ---
# Kita bantu AI dengan mendeskripsikan bentuk batiknya secara harfiah
batik_data = [
    {
        "name": "Megamendung",
        "trigger": "batik_style_megamendung",
        # Deskripsi fisik bentuk awan
        "visual_desc": "(cloud shape pattern:1.3), (curved lines:1.2), blue and red gradient clouds, horizontal cloud lines"
    },
    {
        "name": "Parang",
        "trigger": "batik_style_parang",
        # Deskripsi fisik garis miring
        "visual_desc": "(diagonal parallel lines:1.4), (knife shape pattern:1.2), s-shape interlocking lines, slanted texture"
    },
    {
        "name": "Kawung",
        "trigger": "batik_style_kawung",
        "visual_desc": "(geometric circles:1.3), (palm fruit shape:1.2), four oval shapes, symmetrical grid"
    },
    {
        "name": "Sekar Jagad",
        "trigger": "batik_style_sekar_jagad",
        "visual_desc": "(map like pattern:1.3), (irregular islands:1.2), patchwork of flowers, mosaic style"
    },
    {
        "name": "Truntum",
        "trigger": "batik_style_truntum",
        "visual_desc": "(small stars pattern:1.4), (scattered dots:1.2), jasmine flower shape, tiny white stars on dark background"
    },
     {
        "name": "Tambal",
        "trigger": "batik_style_tambal",
        "visual_desc": "(patchwork pattern:1.4), (triangle and square patches:1.2), diverse textures in grid"
    }
]

# --- 4. FUNGSI GENERATE DENGAN BANTUAN ---
def rescue_mission_batik(seed=42):
    print(f"\n{'='*70}")
    print(f"🚑 MISI PENYELAMATAN: MEMISAHKAN MOTIF (Seed: {seed})")
    print(f"{'='*70}")

    # Kita tes 6 motif utama dulu (2 baris x 3 kolom)
    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    axs = axs.flatten()

    item_name = "Luxury Handbag" # Kita tes di tas

    for i, data in enumerate(batik_data):
        print(f"🔄 ({i+1}/6) Memaksa keluar motif {data['name']}...", end="\r")

        # RAHASIA 1: Gabungkan Trigger + Deskripsi Visual
        final_prompt = f"RAW photo of a {item_name} made of {data['trigger']} fabric, {data['visual_desc']}, intricate details, 8k, realistic"

        # Negative Prompt yang kuat melawan Kawung (Circles) jika bukan Kawung
        neg = "(deformed:1.3), cartoon, low quality"
        if data['name'] != "Kawung":
            neg += ", (circles:1.2), (geometric:1.1)" # Cegah bentuk bulat kalau bukan Kawung

        generator = torch.Generator("cuda").manual_seed(seed)

        # RAHASIA 2: Turunkan Scale LoRA ke 0.65 - 0.7
        # Biar dia 'dengar' deskripsi visual kita, bukan cuma nurut LoRA
        image = pipe(
            final_prompt,
            negative_prompt=neg,
            num_inference_steps=40,
            guidance_scale=7.5,
            cross_attention_kwargs={"scale": 0.65}, # <--- KUNCINYA DI SINI (Jangan 1.0)
            generator=generator
        ).images[0]

        # Tampilkan
        ax = axs[i]
        ax.imshow(image)
        ax.set_title(f"{data['name']}\n(Booster: ON)", fontsize=12, fontweight='bold')
        ax.axis("off")

    plt.tight_layout()
    plt.show()

# Jalankan
rescue_mission_batik(seed=12345) # Ganti seed kalau masih gagal